# Data Downloader and cleaner
This notebook develops a method to clean the database and download data

`meta` refers to the metadata of the audio clips, and is read from TrainingData_BU&Public_CWS.csv

Operations performed:
- Remove samples with abundance labels 'too many to tag'
- Include only vocalizations with type 'song'
- Vocalizations tagged outside of wildtrax aren't included since the start and end times can be inaccurate. 
- The 'tag_rating' field has been left unfiltered.
- Only olive-sided flycatcher vocalizations are included. 
- Convert Id fields from float type to int type using pd.astype()
- Some clips in the .csv file don't have a url to the audio data. These are skipped.
<br>

Todo: 
- Check to make sure that tags with a different tag id but the same recording id do actually refer to the same recording - and not just duplicated recording ids for different recordings. 


In [1]:
import pandas as pd
import requests
import re
from pathlib import Path

In [20]:
data_path = Path('../../data/raw/')

PosixPath('../../data/raw')

In [21]:
meta = pd.read_csv(data_path/'TrainingData_BU&Public_CWS.csv')

/var/folders/hy/36n7jr6x3h74ddx507dc12_00000gn/T/ipykernel_47988/2033733313.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv(data_path/'TrainingData_BU&Public_CWS.csv')


In [22]:
len(meta)

1152840

In [23]:
def display_all(df, rows=10, cols=200):
    with pd.option_context("display.max_columns", cols):
           display(df)

# Clean the data

One of the entries in the .csv file didn't contain Id tags. Remove this from the dataset

In [24]:
meta.drop(index=1152839, inplace=True)

In [25]:
# convert ids to integer types
meta.recording_id = meta.recording_id.astype(int)
meta.location_id = meta.location_id.astype(int)
meta.task_id = meta.task_id.astype(int)
meta.tag_id = meta.tag_id.astype(int)

Some of the abundance types are stored as strings and some are numeric. Convert all to same type.

In [26]:
meta.abundance = meta.abundance.astype(str)

In [27]:
display_all(meta.head())

,organization,project,project_id,location,location_id,recording_date_time,recording_id,task_method,task_id,aru_task_status,species_code,species_common_name,species_scientific_name,species_class,detection_time,task_duration,tag_duration,min_tag_freq,max_tag_freq,tag_id,individual_order,vocalization,abundance,tag_rating,tag_is_verified,clip_channel_used,observer,observer_user_id,verifier_user_id,left_full_freq_tag_rms_peak_dbfs,left_full_freq_tag_rms_trough_dbfs,left_full_freq_tag_pk_count,left_full_freq_tag_dc_offset,left_full_freq_tag_min_level,left_full_freq_tag_max_level,left_full_freq_tag_peak_level_dbfs,left_freq_filter_tag_rms_peak_dbfs,left_freq_filter_tag_rms_trough_dbfs,left_freq_filter_tag_pk_count,left_freq_filter_tag_dc_offset,left_freq_filter_tag_min_level,left_freq_filter_tag_max_level,left_freq_filter_tag_peak_level_dbfs,right_full_freq_tag_rms_peak_dbfs,right_full_freq_tag_rms_trough_dbfs,right_full_freq_tag_pk_count,right_full_freq_tag_dc_offset,right_full_freq_tag_min_level,right_full_freq_tag_max_level,right_full_freq_tag_peak_level_dbfs,right_freq_filter_tag_rms_peak_dbfs,right_freq_filter_tag_rms_trough_dbfs,right_freq_filter_tag_pk_count,right_freq_filter_tag_dc_offset,right_freq_filter_tag_min_level,right_freq_filter_tag_max_level,right_freq_filter_tag_peak_level_dbfs,tagged_in_wildtrax,url,tag_spectrogram_url,clip_url,sensorId,tasks,status
0,BU,Alberta Archetypes,1501,BBS-4139-20,87850,2021-07-03 06:56:02,331758,None,595843,Transcribed,SAVS,Savannah Sparrow,PASSERCULUS SANDWICHENSIS,Aves,10.74,180.0,2.51,2.01kHz,10.09kHz,3212033,1.0,Song,1,NaN,f,1.0,Not Assigned,-1.0,NaN,-41.64,-55.62,2.0,-0.000014,-0.020752,0.017666,-33.66,-42.32,-67.30,2.0,0.0,-0.016110,0.016191,-35.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,https://portal.wildtrax.ca/home/aru-tasks/reco...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,357,Active
1,BU,Alberta Archetypes,1501,BBS-4139-27,87844,2021-07-03 07:42:01,331765,None,595844,Transcribed,SAVS,Savannah Sparrow,PASSERCULUS SANDWICHENSIS,Aves,20.35,180.0,2.51,2.29kHz,10.32kHz,3212034,1.0,Song,1,NaN,f,1.0,Not Assigned,-1.0,NaN,-22.50,-39.54,2.0,-0.000087,-0.137454,0.144287,-16.82,-41.80,-64.05,2.0,0.0,-0.036974,0.035904,-28.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,https://portal.wildtrax.ca/home/aru-tasks/reco...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,357,Active
2,BU,Alberta Archetypes,1501,BBS-4139-28,87842,2021-07-03 07:47:01,331764,None,595845,Transcribed,SAVS,Savannah Sparrow,PASSERCULUS SANDWICHENSIS,Aves,7.18,180.0,2.51,2.19kHz,10.79kHz,3212035,1.0,Song,1,NaN,f,1.0,Not Assigned,-1.0,NaN,-23.62,-52.42,2.0,0.000045,-0.149939,0.135607,-16.48,-51.09,-66.13,2.0,0.0,-0.007927,0.008014,-41.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,https://portal.wildtrax.ca/home/aru-tasks/reco...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,357,Active
3,BU,Alberta Archetypes,1501,BBS-4139-28,87842,2021-07-03 07:47:01,331764,None,595845,Transcribed,SAVS,Savannah Sparrow,PASSERCULUS SANDWICHENSIS,Aves,71.73,180.0,2.51,1.63kHz,10.65kHz,3212036,1.0,Song,1,NaN,f,1.0,Not Assigned,-1.0,NaN,-15.98,-28.24,2.0,-0.000270,-0.317960,0.371485,-8.60,-46.94,-65.00,2.0,0.0,-0.013312,0.013227,-37.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,https://portal.wildtrax.ca/home/aru-tasks/reco...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,357,Active
4,BU,Alberta Archetypes,1501,BBS-4139-36,87834,2022-06-26 08:24:04,358620,None,595846,Transcribed,OVEN,Ovenbird,SEIURUS AUROCAPILLA,Aves,2.75,180.0,2.57,2.16kHz,10.92kHz,3212037,1.0,Song,1,NaN,f,1.0,Not Assigned,-1.0,NaN,-36.45,-50.15,2.0,-0.000026,-0.041443,0.051636,-25.74,-40.94,-57.32,2.0,0.0,-0.033356,0.035675,-28.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [28]:
meta.abundance.value_counts()

1       1024057
nan       49875
TMTT      40077
CI 1      17247
CI 3      11324
CI 2       8411
2          1167
3           342
5           126
4           125
6            36
7            28
8            14
10            9
9             1
Name: abundance, dtype: int64

In [29]:
# Drop tmtt abundance tags.
tmtt_idxs = meta[meta.abundance=='TMTT'].index
meta.drop(tmtt_idxs, inplace=True)

In [30]:
# Drop non song vocalizations
not_song_idxs = meta[meta.vocalization!='Song'].index
meta.drop(not_song_idxs, inplace=True)

In [31]:
# Drop recordings not labeled in wildtrax
labeled_elsewhere_idxs = meta[meta.tagged_in_wildtrax=='f'].index
meta.drop(labeled_elsewhere_idxs, inplace=True)

In [32]:
# choose OSFL entries
osfl_idxs = meta[meta.species_code=='OSFL'].index
osfls = meta.loc[osfl_idxs]


Add a column named `file_type` to the dataframe. This is done only to samples with a clip_url.

In [33]:
osfls['file_type'] = None
for idx in osfls[~osfls.clip_url.isna()].index:
    
    osfls['file_type'][idx] = osfls['clip_url'][idx].split('.')[-1]

/var/folders/hy/36n7jr6x3h74ddx507dc12_00000gn/T/ipykernel_47988/3830306749.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  osfls['file_type'][idx] = osfls['clip_url'][idx].split('.')[-1]


See how the file_type has been added to the meta dataframe as well as the osfls dataframe to understand why the error message above was generated.

In [34]:
meta.head()

,organization,project,project_id,location,location_id,recording_date_time,recording_id,task_method,task_id,aru_task_status,...,right_freq_filter_tag_min_level,right_freq_filter_tag_max_level,right_freq_filter_tag_peak_level_dbfs,tagged_in_wildtrax,url,tag_spectrogram_url,clip_url,sensorId,tasks,status
382,BU,Alberta Archetypes,1501,P-E0-1-10,308678,2022-06-05 06:51:00,416962,None,596169,Transcribed,...,NaN,NaN,NaN,t,https://portal.wildtrax.ca/home/aru-tasks/reco...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,357,Active
425,BU,Amplitude Quality Testing 2020,293,AM-403-SE2,36043,2017-06-15 04:46:00,92051,None,87956,Transcribed,...,-0.069424,0.063139,-23.17,t,https://portal.wildtrax.ca/home/aru-tasks/reco...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,174,Published - Private
427,BU,Amplitude Quality Testing 2020,293,AM-403-SE2,36043,2017-06-15 04:46:00,92051,None,87898,Transcribed,...,-0.040386,0.036900,-27.88,t,https://portal.wildtrax.ca/home/aru-tasks/reco...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,174,Published - Private
428,BU,Amplitude Quality Testing 2020,293,AM-403-SE2,36043,2017-06-15 04:46:00,92051,None,87840,Transcribed,...,-0.064721,0.062714,-23.78,t,https://portal.wildtrax.ca/home/aru-tasks/reco...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,174,Published - Private
429,BU,Amplitude Quality Testing 2020,293,AM-403-SE2,36043,2017-06-15 04:46:00,92051,None,87927,Transcribed,...,-0.037280,0.040078,-27.94,t,https://portal.wildtrax.ca/home/aru-tasks/reco...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,174,Published - Private


# drop entries with a missing clip_url field from OSFLs

In [35]:
len(osfls)

5167

In [36]:
osfls.drop(osfls.loc[osfls.clip_url.isna()].index, inplace=True)

In [37]:
len(osfls)

5163

# Download the audio samples

1. Get the URL from the database (example)

In [38]:
url = osfls.clip_url.iloc[0]

In [39]:
url

'https://wildtrax-aru.s3.us-west-2.amazonaws.com/df7f6bdc-9300-4738-9ddb-81d84d448400/687a4de556e1439fa624365426dcb3b0.mp3'

2. Extract the file type from the url (example)

In [41]:
url.split('.')[-1]

'mp3'

3. Save the remaining list of URLS to disk with the correct file extensions. 

In [43]:
display_all(osfls.head())

,organization,project,project_id,location,location_id,recording_date_time,recording_id,task_method,task_id,aru_task_status,species_code,species_common_name,species_scientific_name,species_class,detection_time,task_duration,tag_duration,min_tag_freq,max_tag_freq,tag_id,individual_order,vocalization,abundance,tag_rating,tag_is_verified,clip_channel_used,observer,observer_user_id,verifier_user_id,left_full_freq_tag_rms_peak_dbfs,left_full_freq_tag_rms_trough_dbfs,left_full_freq_tag_pk_count,left_full_freq_tag_dc_offset,left_full_freq_tag_min_level,left_full_freq_tag_max_level,left_full_freq_tag_peak_level_dbfs,left_freq_filter_tag_rms_peak_dbfs,left_freq_filter_tag_rms_trough_dbfs,left_freq_filter_tag_pk_count,left_freq_filter_tag_dc_offset,left_freq_filter_tag_min_level,left_freq_filter_tag_max_level,left_freq_filter_tag_peak_level_dbfs,right_full_freq_tag_rms_peak_dbfs,right_full_freq_tag_rms_trough_dbfs,right_full_freq_tag_pk_count,right_full_freq_tag_dc_offset,right_full_freq_tag_min_level,right_full_freq_tag_max_level,right_full_freq_tag_peak_level_dbfs,right_freq_filter_tag_rms_peak_dbfs,right_freq_filter_tag_rms_trough_dbfs,right_freq_filter_tag_pk_count,right_freq_filter_tag_dc_offset,right_freq_filter_tag_min_level,right_freq_filter_tag_max_level,right_freq_filter_tag_peak_level_dbfs,tagged_in_wildtrax,url,tag_spectrogram_url,clip_url,sensorId,tasks,status,file_type
3646,BU,ARU Test Project Model Comparisons 2021,1099,ATP-FLB-14A,238593,2021-06-28 06:00:00,366621,1SPM,343288,Transcribed,OSFL,Olive-sided Flycatcher,CONTOPUS COOPERI,Aves,117.83,180.0,1.54,1.03kHz,5.14kHz,2748897,1.0,Song,1,NaN,f,1.0,Marcus Sommers,1224.0,NaN,-35.81,-38.45,2.0,0.013525,0.002960,0.024292,-32.29,-52.74,-68.22,2.0,-0.000001,-0.005737,0.005890,-44.60,-45.89,-52.40,2.0,0.002683,-0.005341,0.012024,-38.40,-58.97,-70.68,2.0,0.000000,-0.003204,0.003113,-49.89,t,https://portal.wildtrax.ca/home/aru-tasks/reco...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,212,Active,mp3
3696,BU,ARU Test Project Model Comparisons 2021,1099,ATP-FLB-14A,238593,2021-07-02 05:00:00,366633,1SPM,343300,Transcribed,OSFL,Olive-sided Flycatcher,CONTOPUS COOPERI,Aves,0.29,180.0,1.58,1.59kHz,4.90kHz,2744797,1.0,Song,1,NaN,f,2.0,Steve Enid,660.0,NaN,-35.45,-38.60,3.0,0.013311,-0.001526,0.028931,-30.77,-63.31,-67.90,2.0,0.000000,-0.005219,0.002075,-45.65,-44.78,-51.87,2.0,0.002186,-0.009827,0.014099,-37.02,-64.91,-70.18,2.0,0.000000,-0.002167,0.002563,-51.82,t,https://portal.wildtrax.ca/home/aru-tasks/reco...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,212,Active,mp3
3740,BU,ARU Test Project Model Comparisons 2021,1099,ATP-FLB-14A,238593,2021-07-02 06:00:00,366638,1SPM,343305,Transcribed,OSFL,Olive-sided Flycatcher,CONTOPUS COOPERI,Aves,126.36,180.0,1.00,2.05kHz,3.78kHz,2748907,2.0,Song,1,NaN,f,2.0,Kelsey Bell,1242.0,NaN,-32.35,-39.48,2.0,0.013461,-0.017395,0.038757,-28.23,-71.09,-73.78,2.0,0.000000,-0.001617,0.001678,-55.50,-35.08,-44.52,2.0,0.002601,-0.024017,0.034393,-29.27,-71.54,-75.24,2.0,0.000000,-0.002899,0.002167,-50.75,t,https://portal.wildtrax.ca/home/aru-tasks/reco...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,212,Active,mp3
3759,BU,ARU Test Project Model Comparisons 2021,1099,ATP-FLB-14B,238600,2021-06-28 05:00:00,366610,1SPM,343277,Transcribed,OSFL,Olive-sided Flycatcher,CONTOPUS COOPERI,Aves,41.15,180.0,1.63,1.63kHz,5.51kHz,2746867,1.0,Song,1,NaN,f,1.0,Steve Enid,660.0,NaN,-22.89,-22.95,2.0,-0.071428,-0.076202,-0.066406,-22.36,-58.90,-66.70,2.0,0.000002,-0.006744,0.024139,-32.35,-25.76,-25.82,4.0,-0.051322,-0.055420,-0.047180,-25.13,-61.10,-67.13,2.0,0.000001,-0.005646,0.017609,-35.09,t,https://portal.wildtrax.ca/home/aru-tasks/reco...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,212,Active,mp3
3877,BU,ARU Test Project Model Compar

# Download a bunch of clips, without duplicating the download if a file with the same name already exists.
- Use relative paths so that the downloader will work on other operating systems within different absolute directories. 
- Loop through all of the clip URLs and build a filename from the recording ID and the file type.
- If a recording with this ID already exists, skip the recording. We already downloaded audio from this recording. 
- Check this assumption in future - it could be the case that these are short clips of audio taken from longer recordings where the longer recordings have unique ids but the clips are all different. 

In [94]:
def exists(fname):
    '''
    check to see whether a file exists
    '''
    return Path.exists(fname)

In [96]:
rec_path = Path.joinpath(data_path, 'recordings')

for rec in (osfls.index):
    clip_url = osfls.clip_url[rec]
    ext = osfls.file_type[rec]
    file = Path.joinpath(rec_path, str(osfls.recording_id[rec]) + '.' + ext)
    
    if exists(file):
        print(f'{file} already downloaded')
    else:
        r = requests.get(clip_url)
        with open(file, 'wb') as f:
            f.write(r.content)

../../data/raw/recordings/366621.mp3 already downloaded
../../data/raw/recordings/366633.mp3 already downloaded
../../data/raw/recordings/366638.mp3 already downloaded
../../data/raw/recordings/366610.mp3 already downloaded
../../data/raw/recordings/366631.flac already downloaded
../../data/raw/recordings/366624.mp3 already downloaded
../../data/raw/recordings/366624.mp3 already downloaded
../../data/raw/recordings/366618.flac already downloaded
../../data/raw/recordings/366627.mp3 already downloaded
../../data/raw/recordings/366637.mp3 already downloaded
../../data/raw/recordings/366637.mp3 already downloaded
../../data/raw/recordings/366643.flac already downloaded
../../data/raw/recordings/366643.flac already downloaded
../../data/raw/recordings/366643.flac already downloaded
../../data/raw/recordings/366643.flac already downloaded
../../data/raw/recordings/366643.flac already downloaded
../../data/raw/recordings/366614.mp3 already downloaded
../../data/raw/recordings/366614.mp3 alre

../../data/raw/recordings/248719.mp3 already downloaded
../../data/raw/recordings/218319.mp3 already downloaded
../../data/raw/recordings/218207.mp3 already downloaded
../../data/raw/recordings/218207.mp3 already downloaded
../../data/raw/recordings/218207.mp3 already downloaded
../../data/raw/recordings/218207.mp3 already downloaded
../../data/raw/recordings/218207.mp3 already downloaded
../../data/raw/recordings/218207.mp3 already downloaded
../../data/raw/recordings/218207.mp3 already downloaded
../../data/raw/recordings/218207.mp3 already downloaded
../../data/raw/recordings/218207.mp3 already downloaded
../../data/raw/recordings/218207.mp3 already downloaded
../../data/raw/recordings/218207.mp3 already downloaded
../../data/raw/recordings/218207.mp3 already downloaded
../../data/raw/recordings/218207.mp3 already downloaded
../../data/raw/recordings/218207.mp3 already downloaded
../../data/raw/recordings/218207.mp3 already downloaded
../../data/raw/recordings/218207.mp3 already dow

# Downloader

In [ ]:
for rec in (osfls.index):
    clip_url=osfls.clip_url[rec]
    ext = osfls.file_type[rec]
    path = 'data/recordings'
    fname = f'{path}/{osfls.recording_id[rec]}.{ext}'
    if exists(fname):
        pass
    else:
        r = requests.get(clip_url)
        with open(fname, 'wb') as f:
            f.write(r.content)